In [2]:
import sklearn.externals 
from time import time
from sklearn import decomposition
from sklearn.impute import KNNImputer
import sklearn.metrics as skm
from skmultilearn.problem_transform import ClassifierChain
import joblib
from sklearn.metrics import make_scorer
from sklearn.model_selection import LeaveOneOut
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from pandas.plotting import scatter_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.stats import norm 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import QuantileTransformer
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import hamming_loss
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import numpy as np
import copy
from sklearn.linear_model import LogisticRegression, SGDClassifier
from skmultiflow.core import BaseSKMObject, ClassifierMixin, MetaEstimatorMixin, MultiOutputMixin
from skmultiflow.utils import check_random_state
import warnings
warnings.filterwarnings("ignore")

In [3]:
#pip install xlrd

In [4]:
#pip install scikit-multiflow - python 3.8

In [5]:
df = pd.read_excel('meta_dataset_n5_k5.xls')
df = df.drop(['data_id','data_name','run_id','task_id','setup_id','flow_id'], axis = 1)
df

,leaves,leaves_branch.mean,leaves_branch.sd,leaves_corrob.mean,leaves_corrob.sd,leaves_homo.mean,leaves_homo.sd,leaves_per_class.mean,leaves_per_class.sd,nodes,...,t1.sd,t2,t3,t4,one-hot-encoder=sklearn.preprocessing.data.OneHotEncoder,variance-thresholding=sklearn.feature_selection.variance_threshold.VarianceThreshold,feature-scaling=sklearn.preprocessing.data.StandardScaler,Classifier_SVC,Classifier_Random,Classifier_Decision
0,2,1.000000,0.000000,0.500000,0.133020,4.000000e+00,0.000000e+00,0.500000,0.000000,1,...,0.247170,0.158416,0.079208,0.500000,0,0,0,0,0,1
1,511,13.414873,4.583014,0.001957,0.003304,1.307730e+07,4.418819e+07,0.100000,0.107707,510,...,0.096821,0.005391,0.004717,0.875000,0,0,0,0,1,0
2,34,6.235294,2.000891,0.029412,0.042136,7.448000e+02,8.714009e+02,0.500000,0.041595,33,...,0.173415,0.164948,0.005155,0.031250,1,1,1,0,0,1
3,12,3.750000,0.753778,0.083333,0.126551,4.613333e+01,1.622867e+01,0.333333,0.166667,11,...,0.030644,0.073034,0.005618,0.076923,1,1,1,1,0,0
4,685,11.281752,2.673954,0.001460,0.005588,6.863978e+05,1.954417e+06,0.500000,0.021678,684,...,0.019709,0.002130,0.001065,0.500000,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,41071,20.970271,5.554298,0.000024,0.000546,1.561971e+17,1.284196e+19,0.500000,0.031593,41070,...,NaN,0.000034,0.000029,0.852941,0,0,0,0,0,1
286,94031,20.639768,4.376983,0.000011,0.000087,2.352799e+13,1.758731e+15,0.500000,0.001557,94030,...,NaN,0.000013,0.000003,0.230769,0,0,0,0,0,1
287,28131,19.268245,4.447649,0.000036,0.000253,4.150850e+11,1.109654e+13,0.142857,0.100286,28130,...,NaN,0.000065,0.000044,0.666667,0,0,0,0,0,1
288,3681,13.742461,2.279269,0.000272,0.000856,8.650965e+06,1.200670e+07,0.333333,0.238048,3680,...,NaN,0.000163,0.000127,0.777778,0,0,0,0,0,1


In [6]:
#exclude meta-features with the percentage of missing values greater than 10%
total = df.isnull().sum().sort_values(ascending=False)
percent_1 = df.isnull().sum() / df.isnull().count()*100
percent_2 = (round(percent_1, 2)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total Nulls', '%'])
missing_data = missing_data.sort_values(by=['Total Nulls'], ascending=False)
missing_data.head(50)

,Total Nulls,%
f2.sd,230,79.31
f3.sd,222,76.55
l2.sd,222,76.55
f4.sd,222,76.55
l1.sd,222,76.55
f1v.sd,222,76.55
l3.sd,222,76.55
MeanNoiseToSignalRatio,212,73.10
Quartile3MutualInformation,210,72.41
MinMutualInformation,210,72.41


In [7]:
df = df.drop(['f2.sd','f3.sd','l2.sd','f4.sd','l1.sd','f1v.sd','l3.sd','MeanNoiseToSignalRatio',
             'Quartile3MutualInformation','MinMutualInformation','Quartile2MutualInformation','Quartile1MutualInformation',
             'Quartile1AttributeEntropy','Quartile3AttributeEntropy','MeanMutualInformation','MaxAttributeEntropy',
             'MeanAttributeEntropy','MinAttributeEntropy','Quartile2AttributeEntropy','MaxMutualInformation',      
             'n2.mean','n2.sd', 'n1','t1.sd','hubs.mean','hubs.sd',
              'n4.sd','n4.mean','t1.mean','density','cls_coef','n3.sd','n3.mean','lsc'
             ], axis = 1)
df

,leaves,leaves_branch.mean,leaves_branch.sd,leaves_corrob.mean,leaves_corrob.sd,leaves_homo.mean,leaves_homo.sd,leaves_per_class.mean,leaves_per_class.sd,nodes,...,l3.mean,t2,t3,t4,one-hot-encoder=sklearn.preprocessing.data.OneHotEncoder,variance-thresholding=sklearn.feature_selection.variance_threshold.VarianceThreshold,feature-scaling=sklearn.preprocessing.data.StandardScaler,Classifier_SVC,Classifier_Random,Classifier_Decision
0,2,1.000000,0.000000,0.500000,0.133020,4.000000e+00,0.000000e+00,0.500000,0.000000,1,...,0.000000,0.158416,0.079208,0.500000,0,0,0,0,0,1
1,511,13.414873,4.583014,0.001957,0.003304,1.307730e+07,4.418819e+07,0.100000,0.107707,510,...,0.046774,0.005391,0.004717,0.875000,0,0,0,0,1,0
2,34,6.235294,2.000891,0.029412,0.042136,7.448000e+02,8.714009e+02,0.500000,0.041595,33,...,0.396907,0.164948,0.005155,0.031250,1,1,1,0,0,1
3,12,3.750000,0.753778,0.083333,0.126551,4.613333e+01,1.622867e+01,0.333333,0.166667,11,...,0.000000,0.073034,0.005618,0.076923,1,1,1,1,0,0
4,685,11.281752,2.673954,0.001460,0.005588,6.863978e+05,1.954417e+06,0.500000,0.021678,684,...,0.100548,0.002130,0.001065,0.500000,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,41071,20.970271,5.554298,0.000024,0.000546,1.561971e+17,1.284196e+19,0.500000,0.031593,41070,...,0.067120,0.000034,0.000029,0.852941,0,0,0,0,0,1
286,94031,20.639768,4.376983,0.000011,0.000087,2.352799e+13,1.758731e+15,0.500000,0.001557,94030,...,0.074462,0.000013,0.000003,0.230769,0,0,0,0,0,1
287,28131,19.268245,4.447649,0.000036,0.000253,4.150850e+11,1.109654e+13,0.142857,0.100286,28130,...,0.102027,0.000065,0.000044,0.666667,0,0,0,0,0,1
288,3681,13.742461,2.279269,0.000272,0.000856,8.650965e+06,1.200670e+07,0.333333,0.238048,3680,...,0.310311,0.000163,0.000127,0.777778,0,0,0,0,0,1


In [8]:
total = df.isnull().sum().sort_values(ascending=False)
percent_1 = df.isnull().sum() / df.isnull().count()*100
percent_2 = (round(percent_1, 2)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total Nulls', '%'])
missing_data = missing_data.sort_values(by=['Total Nulls'], ascending=False)
missing_data.head(40)

,Total Nulls,%
Quartile3SkewnessOfNumericAtts,22,7.59
MaxStdDevOfNumericAtts,22,7.59
MinMeansOfNumericAtts,22,7.59
Quartile2MeansOfNumericAtts,22,7.59
MinStdDevOfNumericAtts,22,7.59
Quartile1StdDevOfNumericAtts,22,7.59
Quartile1SkewnessOfNumericAtts,22,7.59
Quartile1MeansOfNumericAtts,22,7.59
Quartile1KurtosisOfNumericAtts,22,7.59
MinSkewnessOfNumericAtts,22,7.59


In [9]:
columns_to_impute = [
    "Quartile3KurtosisOfNumericAtts",
    "Quartile3MeansOfNumericAtts",
    "Quartile1MeansOfNumericAtts",
    "Quartile1SkewnessOfNumericAtts",
    "Quartile1StdDevOfNumericAtts",
    "Quartile3SkewnessOfNumericAtts",
    "Quartile2StdDevOfNumericAtts",
    "Quartile2SkewnessOfNumericAtts",
    "Quartile2MeansOfNumericAtts",
    "Quartile1KurtosisOfNumericAtts",
    "MaxKurtosisOfNumericAtts",
    "MaxMeansOfNumericAtts",
    "MinMeansOfNumericAtts",
    "Quartile3StdDevOfNumericAtts",
    "Quartile2KurtosisOfNumericAtts",
    "MinSkewnessOfNumericAtts",
    "MinKurtosisOfNumericAtts",
    "MeanStdDevOfNumericAtts",
    "MeanSkewnessOfNumericAtts",
    "MeanMeansOfNumericAtts",
    "MeanKurtosisOfNumericAtts",
    "MaxStdDevOfNumericAtts",
    "MaxSkewnessOfNumericAtts",
    "MinStdDevOfNumericAtts"
]

# Impute missing values with 0 for the specified columns
df[columns_to_impute] = df[columns_to_impute].fillna(0)

In [10]:
total = df.isnull().sum().sort_values(ascending=False)
percent_1 = df.isnull().sum() / df.isnull().count()*100
percent_2 = (round(percent_1, 2)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total Nulls', '%'])
missing_data = missing_data.sort_values(by=['Total Nulls'], ascending=False)
missing_data.head(40)

,Total Nulls,%
f2.mean,16,5.52
f1.sd,11,3.79
f1.mean,11,3.79
nodes_repeated.sd,10,3.45
nodes_per_level.sd,9,3.10
tree_imbalance.sd,9,3.10
nodes_per_attr,0,0.00
leaves_branch.sd,0,0.00
leaves_corrob.mean,0,0.00
leaves_corrob.sd,0,0.00


In [11]:
columns_to_impute = [
'f2.mean','f1.sd', 'f1.mean','nodes_repeated.sd','nodes_per_level.sd','tree_imbalance.sd']
subset_df = df[columns_to_impute]
imputer = KNNImputer(n_neighbors=5)
df[columns_to_impute] = imputer.fit_transform(subset_df)

In [12]:
total = df.isnull().sum().sort_values(ascending=False)
percent_1 = df.isnull().sum() / df.isnull().count()*100
percent_2 = (round(percent_1, 2)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total Nulls', '%'])
missing_data = missing_data.sort_values(by=['Total Nulls'], ascending=False)
missing_data.head(40)

,Total Nulls,%
leaves,0,0.0
nodes_per_inst,0,0.0
leaves_branch.sd,0,0.0
leaves_corrob.mean,0,0.0
leaves_corrob.sd,0,0.0
leaves_homo.mean,0,0.0
leaves_homo.sd,0,0.0
leaves_per_class.mean,0,0.0
leaves_per_class.sd,0,0.0
nodes,0,0.0


In [14]:
X_orig = df.iloc[:,0:86].to_numpy()
print("entry", df.iloc[:,0:86].columns)

Y_orig = df.iloc[:,[86,87,88,89,90,91]].to_numpy()  
print("labels", df.iloc[:,[86,87,88,89,90,91]].columns)

entry Index(['leaves', 'leaves_branch.mean', 'leaves_branch.sd',
       'leaves_corrob.mean', 'leaves_corrob.sd', 'leaves_homo.mean',
       'leaves_homo.sd', 'leaves_per_class.mean', 'leaves_per_class.sd',
       'nodes', 'nodes_per_attr', 'nodes_per_inst', 'nodes_per_level.mean',
       'nodes_per_level.sd', 'nodes_repeated.mean', 'nodes_repeated.sd',
       'tree_depth.mean', 'tree_depth.sd', 'tree_imbalance.mean',
       'tree_imbalance.sd', 'tree_shape.mean', 'tree_shape.sd',
       'var_importance.mean', 'var_importance.sd', 'AutoCorrelation',
       'ClassEntropy', 'Dimensionality', 'MajorityClassPercentage',
       'MajorityClassSize', 'MaxKurtosisOfNumericAtts',
       'MaxMeansOfNumericAtts', 'MaxNominalAttDistinctValues',
       'MaxSkewnessOfNumericAtts', 'MaxStdDevOfNumericAtts',
       'MeanKurtosisOfNumericAtts', 'MeanMeansOfNumericAtts',
       'MeanNominalAttDistinctValues', 'MeanSkewnessOfNumericAtts',
       'MeanStdDevOfNumericAtts', 'MinKurtosisOfNumericAtts',
    

In [15]:
transformer = QuantileTransformer(output_distribution='normal')

In [16]:
class ClassifierChain(BaseSKMObject, ClassifierMixin, MetaEstimatorMixin, MultiOutputMixin):
    """ Classifier Chains for multi-label learning.

    Parameters
    ----------
    base_estimator: skmultiflow.core.BaseSKMObject or sklearn.BaseEstimator (default=LogisticRegression)
        Each member of the ensemble is an instance of the base estimator

    order : str (default=None)
        `None` to use default order, 'random' for random order.

    random_state: int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used by `np.random`.

    Examples
    --------
    >>> from skmultiflow.data import make_logical
    >>>
    >>> X, Y = make_logical(random_state=1)
    >>>
    >>> print("TRUE: ")
    >>> print(Y)
    >>> print("vs")
    >>>
    >>> print("CC")
    >>> cc = ClassifierChain(SGDClassifier(max_iter=100, loss='log', random_state=1))
    >>> cc.fit(X, Y)
    >>> print(cc.predict(X))
    >>>
    >>> print("RCC")
    >>> cc = ClassifierChain(SGDClassifier(max_iter=100, loss='log', random_state=1), order='random', random_state=1)
    >>> cc.fit(X, Y)
    >>> print(cc.predict(X))
    >>>
    TRUE:
    [[1. 0. 1.]
     [1. 1. 0.]
     [0. 0. 0.]
     [1. 1. 0.]]
    vs
    CC
    [[1. 0. 1.]
     [1. 1. 0.]
     [0. 0. 0.]
     [1. 1. 0.]]
    RCC
    [[1. 0. 1.]
     [1. 1. 0.]
     [0. 0. 0.]
     [1. 1. 0.]]



    Notes
    -----
    Classifier Chains [1]_ is a popular method for multi-label learning. It exploits correlation between labels
    by incrementally building binary classifiers for each label.

    scikit-learn also includes 'ClassifierChain'. A difference is probabilistic extensions are included here.


    References
    ----------
    .. [1] Read, Jesse, Bernhard Pfahringer, Geoff Holmes, and Eibe Frank. "Classifier chains for multi-label
       classification." In Joint European Conference on Machine Learning and Knowledge Discovery in Databases,
       pp. 254-269. Springer, Berlin, Heidelberg, 2009.

    """

    # TODO: much of this can be shared with Regressor Chains, probably should use a base class to inherit here.

    def __init__(self, base_estimator=LogisticRegression(), order=None, random_state=None):
        super().__init__()
        self.base_estimator = base_estimator
        self.order = order
        self.random_state = random_state
        self.chain = None
        self.ensemble = None
        self.L = None
        self._random_state = None   # This is the actual random_state object used internally
        self.__configure()

    def __configure(self):
        self.ensemble = None
        self.L = -1
        self._random_state = check_random_state(self.random_state)

    def fit(self, X, y, classes=None, sample_weight=None):
        """ Fit the model.

        Parameters
        ----------
        X : numpy.ndarray of shape (n_samples, n_features)
            The features to train the model.

        y: numpy.ndarray of shape (n_samples, n_targets)
            An array-like with the labels of all samples in X.

        classes: Not used (default=None)

        sample_weight: Not used (default=None)

        Returns
        -------
        self

        """
        N, self.L = y.shape
        L = self.L
        N, D = X.shape

        self.chain = np.arange(L)
        if self.order == 'random':
            self._random_state.shuffle(self.chain)

        # Set the chain order
        y = y[:, self.chain]

        # Train
        self.ensemble = [copy.deepcopy(self.base_estimator) for _ in range(L)]
        XY = np.zeros((N, D + L-1))
        XY[:, 0:D] = X
        XY[:, D:] = y[:, 0:L - 1]
        for j in range(self.L):
            self.ensemble[j].fit(XY[:, 0:D + j], y[:, j])
        return self

    def partial_fit(self, X, y, classes=None, sample_weight=None):
        """ Partially (incrementally) fit the model.

        Parameters
        ----------
        X : numpy.ndarray of shape (n_samples, n_features)
            The features to train the model.

        y: numpy.ndarray of shape (n_samples)
            An array-like with the labels of all samples in X.

        classes: Not used (default=None)

        sample_weight: NOT used (default=None)

        Returns
        -------
        self

        """
        if self.ensemble is None:
            # This is the first time that the model is fit
            self.fit(X, y)
            return self

        N, self.L = y.shape
        L = self.L
        N, D = X.shape

        # Set the chain order
        y = y[:, self.chain]

        XY = np.zeros((N, D + L-1))
        XY[:, 0:D] = X
        XY[:, D:] = y[:, 0:L - 1]
        for j in range(L):
            self.ensemble[j].partial_fit(XY[:, 0:D + j], y[:, j])

        return self

    def predict(self, X):
        """ Predict classes for the passed data.

        Parameters
        ----------
        X : numpy.ndarray of shape (n_samples, n_features)
            The set of data samples to predict the labels for.

        Returns
        -------
        A numpy.ndarray with all the predictions for the samples in X.

        """
        N, D = X.shape
        Y = np.zeros((N, self.L))
        for j in range(self.L):
            if j > 0:
                X = np.column_stack([X, Y[:, j-1]])
            Y[:, j] = self.ensemble[j].predict(X)

        # Unset the chain order (back to default)
        return Y[:, np.argsort(self.chain)]

    def predict_proba(self, X):
        """ Estimates the probability of each sample in X belonging to each of the class-labels.

        Parameters
        ----------
        X : numpy.ndarray of shape (n_samples, n_features)
            The matrix of samples one wants to predict the class probabilities for.

        Returns
        -------
        A numpy.ndarray of shape (n_samples, n_labels), in which each outer entry is associated with the X entry of the
        same index. And where the list in index [i] contains len(self.target_values) elements, each of which represents
        the probability that the i-th sample of X belongs to a certain class-label.

        Notes
        -----
        Returns marginals [P(y_1=1|x),...,P(y_L=1|x,y_1,...,y_{L-1})]
        i.e., confidence predictions given inputs, for each instance.

        This function suitable for multi-label (binary) data
        only at the moment (may give index-out-of-bounds error if
        uni- or multi-target (of > 2 values) data is used in training).
        """
        N, D = X.shape
        Y = np.zeros((N, self.L))
        for j in range(self.L):
            if j > 0:
                X = np.column_stack([X, Y[:, j-1]])
            Y[:, j] = self.ensemble[j].predict_proba(X)[:, 1]
        return Y

    def reset(self):
        self.__configure()
        return self

    def _more_tags(self):
        return {'multioutput': True,
                'multioutput_only': True}


def P(y, x, cc, payoff=np.prod):
    """ Payoff function, P(Y=y|X=x)

        What payoff do we get for predicting y | x, under model cc.

        Parameters
        ----------
        x: input instance
        y: its true labels
        cc: a classifier chain
        payoff: payoff function

        Returns
        -------
        A single number; the payoff of predicting y | x.
    """
    D = len(x)
    L = len(y)

    p = np.zeros(L)
    xy = np.zeros(D + L)
    xy[0:D] = x.copy()
    for j in range(L):
        P_j = cc.ensemble[j].predict_proba(xy[0:D+j].reshape(1, -1))[0]  # e.g., [0.9, 0.1] wrt 0, 1
        xy[D+j] = y[j]                                           # e.g., 1
        p[j] = P_j[y[j]]                                         # e.g., 0.1
                                                                 # or, y[j] = 0 is predicted with probability p[j] = 0.9
    return payoff(p)

In [17]:
#ClassifierChain
from skmultilearn.problem_transform import ClassifierChain
cv = LeaveOneOut()
y_true, y_pred = list(), list()
count = 0
for train_ix, test_ix in cv.split(X_orig):
        count += 1
        X_train, X_test = X_orig[train_ix, :], X_orig[test_ix, :]
        y_train, y_test = Y_orig[train_ix], Y_orig[test_ix]
        X_train_folds = transformer.fit_transform(X_train)
        X_test_folds= transformer.transform(X_test)
        print("Entry", X_test_folds)
        clf = ClassifierChain(DecisionTreeClassifier(criterion='entropy', random_state=17,  min_samples_leaf=10))
        clf.fit(X_train_folds, y_train)
        #clf.base_estimator.fit(X_train_folds, y_train)
        yhat = clf.predict(X_test_folds)
        #print("Output", count,that)
        print("Output", count,yhat.toarray())
        y_true.append(y_test[0])
        y_pred.append(yhat[0].toarray().ravel())

print("f1",f1_score(y_true, y_pred,average='macro'))
print("acc",accuracy_score(y_true, y_pred))
print("precision", precision_score(y_true, y_pred,average='macro'))
print("hamming",hamming_loss(y_true, y_pred))
print(skm.multilabel_confusion_matrix(y_true, y_pred))

Entry [[-5.19933758e+00 -5.19933758e+00 -5.19933758e+00  5.19933758e+00
   1.27132003e+00 -5.19933758e+00 -5.19933758e+00  5.19933758e+00
  -5.19933758e+00 -5.19933758e+00 -2.87416864e+00 -1.41800799e+00
  -5.19933758e+00 -1.83102868e+00 -5.19933758e+00 -1.63272919e+00
  -5.19933758e+00 -5.19933758e+00  5.19933758e+00  2.64832565e-01
   5.19933758e+00 -5.19933758e+00 -3.37007679e-01  1.31328749e+00
   1.78543910e-01 -2.54694753e-01  1.40391085e+00  2.43263067e-01
  -1.41779714e+00 -9.64642612e-01 -1.14953089e-01 -5.19933758e+00
  -8.06570002e-01 -5.48466372e-02 -3.68580607e-01 -3.63067624e-02
  -5.19933758e+00  1.67459499e-02  1.61790463e-02  6.58790415e-01
   8.86987878e-01 -2.01538181e-01  1.49471900e+00  9.87218257e-01
   1.42553491e-01 -9.88481957e-01  2.21282487e+00 -5.19933758e+00
   3.37007679e-01 -1.42576847e+00 -5.19933758e+00 -5.19933758e+00
  -1.36069179e+00  1.69393425e+00  2.67534323e+00 -5.19933758e+00
  -5.19933758e+00 -1.42978834e+00  1.42978834e+00  1.89588223e-01
   6

In [18]:
#LabelPowerset
cv = LeaveOneOut()
y_true, y_pred = list(), list()
count = 0
for train_ix, test_ix in cv.split(X_orig):
        count += 1
        X_train, X_test = X_orig[train_ix, :], X_orig[test_ix, :]
        y_train, y_test = Y_orig[train_ix], Y_orig[test_ix]
        X_train_folds = transformer.fit_transform(X_train)
        X_test_folds= transformer.transform(X_test)
        print("Entry", X_test_folds)
        clf = LabelPowerset(DecisionTreeClassifier(criterion='entropy', random_state=17,  min_samples_leaf=10))
        clf.fit(X_train_folds, y_train)
        yhat = clf.predict(X_test_folds)
        print("Output", count,yhat.toarray())
        y_true.append(y_test[0])
        y_pred.append(yhat[0].toarray().ravel())
print("f1",f1_score(y_true, y_pred,average='macro'))
print("acc",accuracy_score(y_true, y_pred))
print("precision", precision_score(y_true, y_pred,average='macro'))
print("hamming",hamming_loss(y_true, y_pred))
print(skm.multilabel_confusion_matrix(y_true, y_pred))

Entry [[-5.19933758e+00 -5.19933758e+00 -5.19933758e+00  5.19933758e+00
   1.27132003e+00 -5.19933758e+00 -5.19933758e+00  5.19933758e+00
  -5.19933758e+00 -5.19933758e+00 -2.87416864e+00 -1.41800799e+00
  -5.19933758e+00 -1.83102868e+00 -5.19933758e+00 -1.63272919e+00
  -5.19933758e+00 -5.19933758e+00  5.19933758e+00  2.64832565e-01
   5.19933758e+00 -5.19933758e+00 -3.37007679e-01  1.31328749e+00
   1.78543910e-01 -2.54694753e-01  1.40391085e+00  2.43263067e-01
  -1.41779714e+00 -9.64642612e-01 -1.14953089e-01 -5.19933758e+00
  -8.06570002e-01 -5.48466372e-02 -3.68580607e-01 -3.63067624e-02
  -5.19933758e+00  1.67459499e-02  1.61790463e-02  6.58790415e-01
   8.86987878e-01 -2.01538181e-01  1.49471900e+00  9.87218257e-01
   1.42553491e-01 -9.88481957e-01  2.21282487e+00 -5.19933758e+00
   3.37007679e-01 -1.42576847e+00 -5.19933758e+00 -5.19933758e+00
  -1.36069179e+00  1.69393425e+00  2.67534323e+00 -5.19933758e+00
  -5.19933758e+00 -1.42978834e+00  1.42978834e+00  1.89588223e-01
   6

Entry [[-0.26865596 -0.16790399  0.3856048   0.26855104 -0.13376369 -0.01291903
   0.06940913  5.19933758 -0.80017207 -0.26865596  0.12800104  1.27604338
  -0.51650444 -0.42549926  0.10171678  0.5397452  -0.13662531  0.35033302
  -0.21561246  0.45080157  0.33749251  0.63270205  0.83051088  0.58967635
   0.83553721 -0.43346347  0.40027873  0.4239671  -1.02459239 -0.83667055
   0.15941427 -5.19933758 -0.14298652  0.17474452 -0.39117449  0.19979414
  -5.19933758  0.40821095  0.3416602   0.20198653  1.21521826 -0.20153818
   0.58240023  1.56351457 -0.01994179 -0.60506036  1.42978834 -5.19933758
  -0.81828504 -1.00999017 -5.19933758 -5.19933758 -0.83051088  0.86163412
   1.71249462 -5.19933758 -5.19933758 -0.98848196  0.99560019 -0.0232576
   0.99394756  0.10951044  1.34391858 -0.02801285  0.60827515  0.56893772
   0.60203474 -0.41170514  0.37077334  0.25789827  0.39086109 -5.19933758
  -0.24009397  0.358164    0.18660955 -0.14669385  1.10300821  0.82839539
   0.13424111  0.7234282   1.9374

Output 47 [[0 0 0 1 0 0]]
Entry [[-0.03046721 -0.03870789 -0.21446857  0.03039214 -0.37561772 -0.12782209
  -0.19464399  5.19933758  0.51802054 -0.03046721  0.17288083  2.18553942
  -0.02769672 -0.00963466  0.14643445  0.50464173 -0.03913373 -0.23774795
   0.07404453 -0.43358204 -0.01281977  0.41671627  0.63140189  0.77814405
  -0.40524132  0.41225354  0.53843354 -0.45045853 -1.18476314 -0.77043479
   0.27115373 -5.19933758 -3.12054529  0.19989945 -0.21952436  0.44280525
  -5.19933758 -2.33370444  0.40974403  0.71216982  1.65834324 -0.20153818
  -1.03288922  1.59923754  0.90086965 -0.37409541  1.75149251 -5.19933758
  -0.62609901 -1.05954455 -5.19933758 -5.19933758 -1.19358872  1.17602889
   2.03253915 -5.19933758 -5.19933758 -1.11724663  1.11724663  0.26102189
   1.53330313 -1.97071671  1.37424976  0.04444136  0.91709335 -2.5061543
   0.78854169 -0.31609524  0.58594074 -2.87965603  0.44280088 -5.19933758
   0.74225743 -0.74225732 -0.1071349   0.39603396  0.48960122  0.58861395
  -0.64

Output 56 [[0 0 0 0 1 0]]
Entry [[ 0.89037625  0.71831767  0.72861589 -0.89043077 -0.55957179  0.87418753
   0.94460457  5.19933758 -5.19933758  0.89037625  0.41267513 -0.62644871
   0.8130563   0.92325513  0.41267513  0.34092927  0.71817982  0.75545646
  -0.90001859 -0.87614172 -0.7939513  -0.82874986 -0.80017207  0.11938421
  -0.74040218  0.77769126 -0.51841799 -0.83785406  0.67732529 -1.70832166
  -0.18818602 -5.19933758 -1.06024561  0.42121303 -1.5356535  -0.13630965
  -5.19933758 -0.96893622  0.27250481 -0.29375419 -1.62502715 -0.20153818
   0.43133049 -0.09245133  1.53683021  1.14598729  0.03046721 -5.19933758
   0.80618031  0.74159404 -5.19933758 -5.19933758  0.88714656 -5.19933758
  -0.38813428 -5.19933758 -5.19933758  0.93999091 -0.93324314 -0.43192906
  -0.20875438  0.09473675 -0.36324664 -1.19735488 -0.18602808 -0.77991761
   0.16608247 -1.66089782 -0.09314051 -1.06655009 -0.03616116 -5.19933758
   1.38544084 -1.34215357  1.68010508 -1.00470036  1.33590731  1.4697142
   0.56

Output 65 [[0 0 0 0 0 1]]
Entry [[-0.17938057  0.26201893  0.50956222  0.1838047   0.62990343  0.15679352
   0.18865162  5.19933758  2.20901046 -0.17938057 -0.76762781 -1.48971317
  -0.52375863 -0.39456485 -0.6992833  -0.49868898  0.27025856  0.43036472
  -0.53068454  1.45097609 -0.09771226  1.28151705 -0.8679592  -0.8944963
   1.31144417 -5.19933758 -0.38651497  5.19933758  0.79851764  2.03499919
   1.11274882 -5.19933758  2.08849358  1.43476905  2.21719733  0.62918319
  -5.19933758  2.63756386  1.22068197  0.35993611 -0.02881782 -0.20153818
  -1.87472148 -0.4290857  -1.67420427 -1.41384503  0.03046721 -5.19933758
   0.8743192   0.63980742 -5.19933758 -5.19933758  0.93999091 -5.19933758
  -0.42119717 -5.19933758 -5.19933758  0.98141346 -0.98141346  2.49692175
   0.41636592  2.40735302  0.64148657  5.19933758  0.12222279  5.19933758
   0.46628417  2.6736313   0.10760876  2.70052807  0.54447432 -5.19933758
  -5.19933758  5.19933758  0.59275987 -1.08192081 -1.64966954 -5.19933758
  -0.97

...

Entry [[ 1.56626113  1.4433162   1.47761503 -1.56659727 -1.30999473  2.04056611
   2.03827383 -1.11724663  0.3062476   1.56626113  1.8289967  -0.63961933
   1.50300996  1.53136344  1.8289967   1.33863285  1.44329024  1.49234932
  -2.07501304 -1.61162329 -1.57295243 -1.43412191  0.03482132 -1.75157962
  -1.71249462  1.46705494 -2.2522646  -1.4930784   1.71249462 -0.24213213
  -0.25404889  0.69650742 -0.63429784 -1.14403597  0.28453531 -0.1787735
   1.01726417 -1.34066618 -1.16830877  0.86579882  0.30208416  1.41779714
  -0.5969496  -0.26761676 -0.80618031  1.57792286 -5.19933758  1.13364272
  -0.03046721  1.75149251 -5.19933758 -5.19933758  0.17054273 -5.19933758
  -5.19933758 -5.19933758 -5.19933758  0.26414698 -0.25964336  0.49620178
   0.19404311 -1.0039993  -0.63565802  0.52822581 -0.07998009 -0.25814735
  -1.02698991  0.10770903 -0.16580198 -0.46380486 -1.10453795 -5.19933758
  -0.12653865 -0.06532321  0.06063587 -0.2917466   0.32643409 -0.44054605
   0.95358369  1.1973592   0.9178

Entry [[ 6.38246101e-01  1.26169894e+00  2.33204723e+00 -6.38782061e-01
   4.47263015e-02  1.24242155e+00  1.23479992e+00 -2.80820281e+00
   3.15166765e-01  6.38246101e-01  4.08432098e-03 -2.53643404e+00
   3.09495750e-01  1.91699102e-01  2.61136788e-02  5.19299085e-01
   1.26323253e+00  2.32892947e+00 -1.45516285e+00 -6.81786198e-01
  -7.48273004e-01 -4.24740456e-01 -1.08954270e+00  2.02803600e-01
   1.73915319e+00  1.12834145e+00 -1.23071415e+00  8.83433567e-02
   1.58893512e+00  5.19933758e+00  1.17835326e+00  2.55465689e+00
   5.19933758e+00  2.71591599e+00  5.19933758e+00  7.47523278e-01
   2.00262269e+00  5.19933758e+00  1.95077083e+00 -1.90457105e+00
  -2.46163647e-01 -5.19933758e+00 -1.84475105e+00 -5.19933758e+00
  -3.07467934e+00 -5.19933758e+00  1.62498072e+00  2.79385806e+00
   1.08925451e+00  1.36198905e+00 -5.19933758e+00 -5.19933758e+00
   9.08856409e-01  1.41779714e+00  2.64146977e-01 -5.19933758e+00
  -5.19933758e+00 -3.74095410e-01  3.74095410e-01  1.13774776e+00
  -1

Entry [[ 1.69941663  1.79876708  1.92014146 -1.69991993 -1.97211861  1.92118784
   1.94123402  5.19933758 -1.12680722  1.69941663  1.44162705 -0.37690438
   1.60088661  1.62592685  1.43934691  1.2046716   1.79875483  1.92032407
  -1.94447954 -1.79516602 -1.69169527 -1.3720449  -1.77203415 -1.52730897
  -0.65782506  0.36523664 -1.32842734 -0.40271422  2.37796818  0.4695227
  -0.22817162 -5.19933758  0.59303902 -1.28704377  0.43315737 -0.26036332
  -5.19933758  0.70999587 -1.29967349  0.17118985 -0.08094736 -0.20153818
  -0.98150401 -1.15107188  0.83727813  2.60760515  0.03046721 -5.19933758
   1.77203415  1.75149251 -5.19933758 -5.19933758  1.83861236 -5.19933758
  -0.8122178  -5.19933758 -5.19933758  1.83861236 -1.83861236  0.19306546
  -0.16723894  1.04063434 -1.28167249  0.66398691 -0.197127    0.87094708
  -1.17882621  0.46427105 -0.26661084  0.56691563 -1.19890481 -5.19933758
   0.68600089 -0.68600085  0.361467   -0.48224758 -0.11909961 -0.58157559
   0.61152863  1.37234518  0.0902

Entry [[ 1.92148812  2.00028334  1.16049926 -1.92219528 -1.88108242  1.57773669
   1.59391375 -0.85534332  1.96438862  1.92148812  1.75722246  0.04788877
   1.951459    1.97251566  1.75722246  2.37324201  2.00026543  1.16479798
  -1.57541812 -1.4629783  -1.96850968 -1.3944184  -0.39283081 -0.752359
  -1.14241789  1.0416706  -1.64149625 -0.29587389  2.55436135  0.76027487
  -0.13048464  1.09320101  0.90361265 -0.09013461  0.85737736 -0.05204678
   0.48877641  0.98563003 -0.09362833  0.63506827  0.68970874 -0.20153818
   1.66855584 -0.17337597 -1.24496356  1.3393807   1.91450583  0.86163412
   0.40225007  1.75149251 -5.19933758 -5.19933758 -0.83051088  1.69393425
   1.50647826 -5.19933758 -5.19933758 -1.3180109   1.3180109   0.16068228
   0.45141602  0.87802959 -0.55722831  0.597122    0.17551094  0.99106064
  -0.70440869  1.03878714 -0.03358444  1.22204201  0.03818993  1.13364272
  -0.79317108  0.58889289 -0.83857694  0.84449162 -1.10383713  1.41492467
   1.28552863  1.4503756  -0.84819

Entry [[ 1.20641227  1.06338505 -0.2671282  -1.20847629 -1.08146897  0.93416864
   0.81026284  5.19933758  0.83833341  1.20641227  1.20641947  0.19876951
   1.22378084  1.23517164  1.20578491  1.271198    1.0633384  -0.15603277
  -0.52119562 -1.87601234 -1.21415488 -1.01942345  0.03482132  0.26801638
  -0.80184671  0.91789443 -1.10916012 -1.01383036  1.13354931 -3.16955416
  -0.39662689 -5.19933758 -1.11700899 -1.0099936  -2.70626605 -0.34745369
  -5.19933758 -0.9947464  -0.9702115  -2.199364   -1.35381073 -0.20153818
   0.53908646 -0.05093867  1.98833349  1.38302154  0.03046721 -5.19933758
  -0.03046721  1.15882438 -5.19933758 -5.19933758  0.17054273 -5.19933758
   0.1397103  -5.19933758 -5.19933758  0.26414698 -0.25964336 -2.12716477
  -1.51402044  0.09749665 -0.33631111 -2.59892574 -0.3405391  -0.76053986
  -0.7359913  -2.63717012 -0.37601575 -1.11612903 -0.9365541  -5.19933758
   1.80071494 -1.74811685 -0.18404983  0.69937917 -0.43478398  5.19933758
   0.70206458  5.19933758 -0.163

In [19]:
#https://github.com/scikit-multiflow/scikit-multiflow/blob/a7e316d/src/skmultiflow/meta/classifier_chains.py#L273
import numpy as np
import copy
from sklearn.linear_model import LogisticRegression, SGDClassifier
from skmultiflow.core import BaseSKMObject, ClassifierMixin, MetaEstimatorMixin, MultiOutputMixin
from skmultiflow.utils import check_random_state

class ClassifierChain(BaseSKMObject, ClassifierMixin, MetaEstimatorMixin, MultiOutputMixin):
    """ Classifier Chains for multi-label learning.

    Parameters
    ----------
    base_estimator: skmultiflow.core.BaseSKMObject or sklearn.BaseEstimator (default=LogisticRegression)
        Each member of the ensemble is an instance of the base estimator

    order : str (default=None)
        `None` to use default order, 'random' for random order.

    random_state: int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used by `np.random`.

    Examples
    --------
    >>> from skmultiflow.data import make_logical
    >>>
    >>> X, Y = make_logical(random_state=1)
    >>>
    >>> print("TRUE: ")
    >>> print(Y)
    >>> print("vs")
    >>>
    >>> print("CC")
    >>> cc = ClassifierChain(SGDClassifier(max_iter=100, loss='log', random_state=1))
    >>> cc.fit(X, Y)
    >>> print(cc.predict(X))
    >>>
    >>> print("RCC")
    >>> cc = ClassifierChain(SGDClassifier(max_iter=100, loss='log', random_state=1), order='random', random_state=1)
    >>> cc.fit(X, Y)
    >>> print(cc.predict(X))
    >>>
    TRUE:
    [[1. 0. 1.]
     [1. 1. 0.]
     [0. 0. 0.]
     [1. 1. 0.]]
    vs
    CC
    [[1. 0. 1.]
     [1. 1. 0.]
     [0. 0. 0.]
     [1. 1. 0.]]
    RCC
    [[1. 0. 1.]
     [1. 1. 0.]
     [0. 0. 0.]
     [1. 1. 0.]]



    Notes
    -----
    Classifier Chains [1]_ is a popular method for multi-label learning. It exploits correlation between labels
    by incrementally building binary classifiers for each label.

    scikit-learn also includes 'ClassifierChain'. A difference is probabilistic extensions are included here.


    References
    ----------
    .. [1] Read, Jesse, Bernhard Pfahringer, Geoff Holmes, and Eibe Frank. "Classifier chains for multi-label
       classification." In Joint European Conference on Machine Learning and Knowledge Discovery in Databases,
       pp. 254-269. Springer, Berlin, Heidelberg, 2009.

    """

    # TODO: much of this can be shared with Regressor Chains, probably should use a base class to inherit here.

    def __init__(self, base_estimator=LogisticRegression(), order=None, random_state=None):
        super().__init__()
        self.base_estimator = base_estimator
        self.order = order
        self.random_state = random_state
        self.chain = None
        self.ensemble = None
        self.L = None
        self._random_state = None   # This is the actual random_state object used internally
        self.__configure()

    def __configure(self):
        self.ensemble = None
        self.L = -1
        self._random_state = check_random_state(self.random_state)

    def fit(self, X, y, classes=None, sample_weight=None):
        """ Fit the model.

        Parameters
        ----------
        X : numpy.ndarray of shape (n_samples, n_features)
            The features to train the model.

        y: numpy.ndarray of shape (n_samples, n_targets)
            An array-like with the labels of all samples in X.

        classes: Not used (default=None)

        sample_weight: Not used (default=None)

        Returns
        -------
        self

        """
        N, self.L = y.shape
        L = self.L
        N, D = X.shape

        self.chain = np.arange(L)
        if self.order == 'random':
            self._random_state.shuffle(self.chain)

        # Set the chain order
        y = y[:, self.chain]

        # Train
        self.ensemble = [copy.deepcopy(self.base_estimator) for _ in range(L)]
        XY = np.zeros((N, D + L-1))
        XY[:, 0:D] = X
        XY[:, D:] = y[:, 0:L - 1]
        for j in range(self.L):
            self.ensemble[j].fit(XY[:, 0:D + j], y[:, j])
        return self

    def partial_fit(self, X, y, classes=None, sample_weight=None):
        """ Partially (incrementally) fit the model.

        Parameters
        ----------
        X : numpy.ndarray of shape (n_samples, n_features)
            The features to train the model.

        y: numpy.ndarray of shape (n_samples)
            An array-like with the labels of all samples in X.

        classes: Not used (default=None)

        sample_weight: NOT used (default=None)

        Returns
        -------
        self

        """
        if self.ensemble is None:
            # This is the first time that the model is fit
            self.fit(X, y)
            return self

        N, self.L = y.shape
        L = self.L
        N, D = X.shape

        # Set the chain order
        y = y[:, self.chain]

        XY = np.zeros((N, D + L-1))
        XY[:, 0:D] = X
        XY[:, D:] = y[:, 0:L - 1]
        for j in range(L):
            self.ensemble[j].partial_fit(XY[:, 0:D + j], y[:, j])

        return self

    def predict(self, X):
        """ Predict classes for the passed data.

        Parameters
        ----------
        X : numpy.ndarray of shape (n_samples, n_features)
            The set of data samples to predict the labels for.

        Returns
        -------
        A numpy.ndarray with all the predictions for the samples in X.

        """
        N, D = X.shape
        Y = np.zeros((N, self.L))
        for j in range(self.L):
            if j > 0:
                X = np.column_stack([X, Y[:, j-1]])
            Y[:, j] = self.ensemble[j].predict(X)

        # Unset the chain order (back to default)
        return Y[:, np.argsort(self.chain)]

    def predict_proba(self, X):
        """ Estimates the probability of each sample in X belonging to each of the class-labels.

        Parameters
        ----------
        X : numpy.ndarray of shape (n_samples, n_features)
            The matrix of samples one wants to predict the class probabilities for.

        Returns
        -------
        A numpy.ndarray of shape (n_samples, n_labels), in which each outer entry is associated with the X entry of the
        same index. And where the list in index [i] contains len(self.target_values) elements, each of which represents
        the probability that the i-th sample of X belongs to a certain class-label.

        Notes
        -----
        Returns marginals [P(y_1=1|x),...,P(y_L=1|x,y_1,...,y_{L-1})]
        i.e., confidence predictions given inputs, for each instance.

        This function suitable for multi-label (binary) data
        only at the moment (may give index-out-of-bounds error if
        uni- or multi-target (of > 2 values) data is used in training).
        """
        N, D = X.shape
        Y = np.zeros((N, self.L))
        for j in range(self.L):
            if j > 0:
                X = np.column_stack([X, Y[:, j-1]])
            Y[:, j] = self.ensemble[j].predict_proba(X)[:, 1]
        return Y

    def reset(self):
        self.__configure()
        return self

    def _more_tags(self):
        return {'multioutput': True,
                'multioutput_only': True}


def P(y, x, cc, payoff=np.prod):
    """ Payoff function, P(Y=y|X=x)

        What payoff do we get for predicting y | x, under model cc.

        Parameters
        ----------
        x: input instance
        y: its true labels
        cc: a classifier chain
        payoff: payoff function

        Returns
        -------
        A single number; the payoff of predicting y | x.
    """
    D = len(x)
    L = len(y)

    p = np.zeros(L)
    xy = np.zeros(D + L)
    xy[0:D] = x.copy()
    for j in range(L):
        P_j = cc.ensemble[j].predict_proba(xy[0:D+j].reshape(1, -1))[0]  # e.g., [0.9, 0.1] wrt 0, 1
        xy[D+j] = y[j]                                           # e.g., 1
        p[j] = P_j[y[j]]                                         # e.g., 0.1
                                                                 # or, y[j] = 0 is predicted with probability p[j] = 0.9
    return payoff(p)

In [22]:
#Rank 3
class ProbabilisticClassifierChain(ClassifierChain):
    #order=[6,4,1,5,2,3]
    def __init__(self, base_estimator=LogisticRegression(), order=None, random_state=None):
            super().__init__(base_estimator=base_estimator, order=order, random_state=random_state)
            
    def predict(self, X, num_top_probs=3):
        N, D = X.shape
        Yp = np.zeros((N, self.L))
        top_probs = []  
        for n in range(N):          
            label_probs = []          
            for b in range(2 ** self.L):
                y_ = np.array(list(map(int, np.binary_repr(b, width=self.L))))           
                w_ = P(y_, X[n], self)                
                label_probs.append((y_, w_))       
            label_probs.sort(key=lambda x: x[1], reverse=True)           
            top_probs.append(label_probs[:num_top_probs])
            Yp[n, :] = label_probs[0][0]
        return Yp, top_probs

In [23]:
predict_times = []
cv = LeaveOneOut()
y_true, y_pred = list(), list()
count = 0
for train_ix, test_ix in cv.split(X_orig):
        count += 1
        X_train, X_test = X_orig[train_ix, :], X_orig[test_ix, :]
        y_train, y_test = Y_orig[train_ix], Y_orig[test_ix]
        X_train_folds = transformer.fit_transform(X_train) 
        X_test_folds= transformer.transform(X_test) 
        print("Entry", X_test_folds)
        clf = ProbabilisticClassifierChain(DecisionTreeClassifier(criterion='entropy',random_state=17, min_samples_leaf=10                                              ))
        clf.fit(X_train_folds, y_train)
        predict_start_time = time()
        yhat = clf.predict(X_test_folds)
        predict_end_time = time()
        predict_duration = predict_end_time - predict_start_time
        predict_times.append(predict_duration)
        print("Output", count,yhat)
        y_true.append(y_test[0])
        y_pred.append(yhat[0].ravel())
        print("Time:",predict_times)
print("Mean Time:", sum(predict_times) / len(predict_times))
print("f1",f1_score(y_true, y_pred,average='macro'))
print("acc",accuracy_score(y_true, y_pred))
print("precision", precision_score(y_true, y_pred,average='macro'))
print("hamming",hamming_loss(y_true, y_pred))
print(skm.multilabel_confusion_matrix(y_true, y_pred))

Entry [[-5.19933758e+00 -5.19933758e+00 -5.19933758e+00  5.19933758e+00
   1.27132003e+00 -5.19933758e+00 -5.19933758e+00  5.19933758e+00
  -5.19933758e+00 -5.19933758e+00 -2.87416864e+00 -1.41800799e+00
  -5.19933758e+00 -1.83102868e+00 -5.19933758e+00 -1.63272919e+00
  -5.19933758e+00 -5.19933758e+00  5.19933758e+00  2.64832565e-01
   5.19933758e+00 -5.19933758e+00 -3.37007679e-01  1.31328749e+00
   1.78543910e-01 -2.54694753e-01  1.40391085e+00  2.43263067e-01
  -1.41779714e+00 -9.64642612e-01 -1.14953089e-01 -5.19933758e+00
  -8.06570002e-01 -5.48466372e-02 -3.68580607e-01 -3.63067624e-02
  -5.19933758e+00  1.67459499e-02  1.61790463e-02  6.58790415e-01
   8.86987878e-01 -2.01538181e-01  1.49471900e+00  9.87218257e-01
   1.42553491e-01 -9.88481957e-01  2.21282487e+00 -5.19933758e+00
   3.37007679e-01 -1.42576847e+00 -5.19933758e+00 -5.19933758e+00
  -1.36069179e+00  1.69393425e+00  2.67534323e+00 -5.19933758e+00
  -5.19933758e+00 -1.42978834e+00  1.42978834e+00  1.89588223e-01
   6